In [1]:
import os
import pandas as pd
import pyarrow as pa
print(pd.__version__, pa.__version__)

2.3.3 22.0.0


In [2]:
## change working directory
os.chdir('/app')
## show current working directory
os.getcwd()

'/app'

In [3]:
import pyarrow.dataset as ds
import pyarrow.fs as fs
import pyarrow.parquet as pq
from ml.utils.spark_utils import get_spark_session

In [4]:
# Create an S3-compatible filesystem for MinIO
s3 = fs.S3FileSystem(
    access_key="minioadmin",
    secret_key="minioadmin",
    endpoint_override="http://minio:9000",  # MinIO endpoint
)

In [5]:
file_path = "recommendation-system/data/silver/netflix_user_data.parquet"
dataset = ds.dataset(file_path, filesystem=s3, format="parquet")

In [6]:
pq.read_table(
    source=file_path,
    filesystem=s3
)

pyarrow.Table
UserID: string
Rating: int32
Date: date32[day]
MovieID: string
----
UserID: [["2385003","659432","751812","2625420","1650301",...,"2276456","2168452","2058760","1150636","1943364"],["802266","1437916","2405091","1387820","1756839",...,"2604811","559646","1664495","602","1503774"],...,["842798","33453","1267315","2463950","36032",...,"1085610","1327322","1482158","142013","2374784"],["2328565","141601","2463152","1374832","70565",...,"2420260","761176","459277","2407365","627867"]]
Rating: [[4,3,2,2,1,...,3,5,5,4,3],[4,3,5,3,5,...,5,5,5,5,4],...,[4,3,2,1,2,...,4,3,5,3,5],[3,4,3,5,5,...,1,3,3,4,3]]
Date: [[2004-07-08,2005-03-16,2002-12-16,2004-05-25,2005-08-30,...,2005-10-10,2005-10-11,2005-10-15,2005-10-19,2005-10-22],[2005-10-24,2005-10-25,2005-10-25,2005-10-26,2005-10-27,...,2005-06-18,2005-06-20,2005-06-20,2005-06-20,2005-06-21],...,[2004-08-26,2004-06-22,2003-12-16,2005-08-02,2005-08-03,...,2005-06-09,2005-06-13,2005-06-14,2005-06-19,2005-06-21],[2005-06-23,2005-06-26,

In [7]:
for batch in dataset.to_batches(batch_size=100_000):
    df = batch.to_pandas()
    print(df.shape)
    print(df.dtypes)
    break

(100000, 4)
UserID     object
Rating      int32
Date       object
MovieID    object
dtype: object


In [8]:
spark = get_spark_session()

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2e2b31e7-31bf-4783-ae8d-08800af57ace;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.4!hadoop-aws.jar (206ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.262!aws-java-sdk-bundle.jar (26296ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-

In [9]:
df = spark.read.parquet(f"s3a://{file_path}")

25/12/07 23:00:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/12/07 23:00:42 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
df.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- MovieID: string (nullable = true)



In [11]:
df.show(20)

+-------+------+----------+-------+
| UserID|Rating|      Date|MovieID|
+-------+------+----------+-------+
|2385003|     4|2004-07-08|  13368|
| 659432|     3|2005-03-16|  13368|
| 751812|     2|2002-12-16|  13368|
|2625420|     2|2004-05-25|  13368|
|1650301|     1|2005-08-30|  13368|
|2269227|     4|2005-10-27|  13368|
|2220672|     4|2002-08-19|  13368|
|2500511|     4|2003-08-11|  13368|
|1452058|     2|2005-01-29|  13368|
|1624891|     3|2002-07-27|  13368|
| 970031|     3|2004-04-14|  13368|
| 345673|     4|2005-04-07|  13368|
|1426869|     5|2005-04-21|  13368|
|1037088|     2|2005-09-14|  13368|
|2079559|     5|2005-10-08|  13368|
|2175560|     5|2005-01-12|  13368|
| 946805|     4|2005-02-09|  13368|
| 767843|     1|2005-05-04|  13368|
| 464031|     3|2001-02-08|  13368|
|2473764|     4|2005-09-26|  13368|
+-------+------+----------+-------+
only showing top 20 rows



In [12]:
df.select("MovieID").distinct().show(50)

+-------+
|MovieID|
+-------+
|  13442|
|  13610|
|  13772|
|  13865|
|  14157|
|  14204|
|  14369|
|  14838|
|  14887|
|  14899|
|  15269|
|  15271|
|  15555|
|  15574|
|  15634|
|  13985|
|  14218|
|  14879|
|  14974|
|  15052|
|  15448|
|  15539|
|  13395|
|  13460|
|  13660|
|  13866|
|  14000|
|  14455|
|  14525|
|  14552|
|  15041|
|  15273|
|  15420|
|  15435|
|  15478|
|  13805|
|  14313|
|  14340|
|  14392|
|  14439|
|  14649|
|  14697|
|  14839|
|  14940|
|  14963|
|  15176|
|  15277|
|  15433|
|  13429|
|  13655|
+-------+
only showing top 50 rows



In [13]:
df.describe().show()

+-------+------------------+------------------+-----------------+
|summary|            UserID|            Rating|          MovieID|
+-------+------------------+------------------+-----------------+
|  count|         100480507|         100480507|        100480507|
|   mean|1322488.5067232791| 3.604289964420661|9070.914762472286|
| stddev| 764536.8002745666|1.0852185646295625|5131.890697185878|
|    min|                10|                 1|                1|
|    max|            999988|                 5|             9999|
+-------+------------------+------------------+-----------------+



In [14]:
spark.stop()